In [7]:
from google.cloud import aiplatform
import pandas as pd

In [81]:
from google.cloud.aiplatform import Endpoint

In [1]:
# Use this line so we do not need to explicitly specify the project number and region whenever we use AI Platform (Vertex AI) services
aiplatform.init(project='united-impact-363519', location='us-central1')

# Importing model artifacts
model = aiplatform.Model.upload(
    display_name = 'Churn Model',
    description = 'Churn Classification Model',
    artifact_uri = 'gs://churn_data_363519/churn-prediction-pipeline-20230405185731',
    serving_container_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest'
)

Creating Model
Create Model backing LRO: projects/134756275535/locations/us-central1/models/4035517736216952832/operations/4249572652777209856
Model created. Resource name: projects/134756275535/locations/us-central1/models/4035517736216952832@1
To use this Model in another session:
model = aiplatform.Model('projects/134756275535/locations/us-central1/models/4035517736216952832@1')


In [4]:
# optional code to create an endpoint
endpoint = aiplatform.Endpoint.create(display_name = 'Churn Model Endpoint', 
                                      project = 'united-impact-363519', 
                                      location = 'us-central1')

Creating Endpoint
Create Endpoint backing LRO: projects/134756275535/locations/us-central1/endpoints/3280815153934761984/operations/6626347356121989120
Endpoint created. Resource name: projects/134756275535/locations/us-central1/endpoints/3280815153934761984
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/134756275535/locations/us-central1/endpoints/3280815153934761984')


In [70]:
model

resource name: projects/134756275535/locations/us-central1/models/4035517736216952832

In [7]:
# This took 11 minutes
model.deploy(endpoint = endpoint,
            #  service_account = "134756275535-compute@developer.gserviceaccount.com",
             machine_type = 'n1-standard-2'
)

Deploying model to Endpoint : projects/134756275535/locations/us-central1/endpoints/3280815153934761984
Deploy Endpoint model backing LRO: projects/134756275535/locations/us-central1/endpoints/3280815153934761984/operations/751401642217177088
Endpoint model deployed. Resource name: projects/134756275535/locations/us-central1/endpoints/3280815153934761984


resource name: projects/134756275535/locations/us-central1/endpoints/3280815153934761984

In [78]:
endpoint

resource name: projects/134756275535/locations/us-central1/endpoints/3280815153934761984

In [92]:
endpoint.list_models()

[]

In [68]:
# You pay for each model deployed to an endpoint, even if no prediction is made. You must undeploy your model to stop incurring further charges. Models that are not deployed or have failed to deploy are not charged.
endpoint.undeploy(deployed_model_id="5501571356321382400")

Undeploying Endpoint model: projects/134756275535/locations/us-central1/endpoints/3280815153934761984
Undeploy Endpoint model backing LRO: projects/134756275535/locations/us-central1/endpoints/3280815153934761984/operations/5529720846857273344
Endpoint model undeployed. Resource name: projects/134756275535/locations/us-central1/endpoints/3280815153934761984


# Get predict data and predict

In [9]:
input_df = pd.read_csv("../data/bank_data_w6.csv")
X = pd.DataFrame()
# cols to keep
cols = [
    'Customer_Age',
    'Dependent_count',
    'Months_on_book',
    'Total_Relationship_Count',
    'Months_Inactive_12_mon',
    'Contacts_Count_12_mon',
    'Credit_Limit',
    'Total_Revolving_Bal',
    'Avg_Open_To_Buy',
    'Total_Amt_Chng_Q4_Q1',
    'Total_Trans_Amt',
    'Total_Trans_Ct',
    'Total_Ct_Chng_Q4_Q1',
    'Avg_Utilization_Ratio',
]
X[cols] = input_df[cols]

In [10]:
sample_customers = X.sample(2)
sample_customers.to_csv('predict_data/sample_customers.csv', index=False)

In [106]:
sample_customers

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
9603,43,3,28,2,1,3,9959.0,734,9225.0,0.860,16732,116,0.812,0.074
9321,55,2,36,5,3,2,12494.0,0,12494.0,0.576,7026,69,0.917,0.000


In [64]:
input_df.iloc[[3458, 9500]]

,Unnamed: 0,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
3458,3458,710521383,Existing Customer,50,M,3,Uneducated,Single,$80K - $120K,Blue,...,2,0,9810.0,1838,7972.0,0.661,2247,41,0.640,0.187
9500,9500,788908308,Existing Customer,46,M,2,High School,Single,$40K - $60K,Blue,...,3,2,5972.0,0,5972.0,0.606,14227,116,0.706,0.000


In [58]:
sample_customers.values.tolist()

[[50.0,
  3.0,
  38.0,
  6.0,
  2.0,
  0.0,
  9810.0,
  1838.0,
  7972.0,
  0.6609999999999999,
  2247.0,
  41.0,
  0.64,
  0.187],
 [46.0,
  2.0,
  32.0,
  2.0,
  3.0,
  2.0,
  5972.0,
  0.0,
  5972.0,
  0.606,
  14227.0,
  116.0,
  0.706,
  0.0]]

In [60]:
# NOTE: IMPORTANT, Predict
prediction = endpoint.predict(instances=sample_customers.values.tolist())

In [93]:
prediction

Prediction(predictions=[0.0, 0.0], deployed_model_id='5501571356321382400', model_version_id='1', model_resource_name='projects/134756275535/locations/us-central1/models/4035517736216952832', explanations=None)

In [94]:
prediction.predictions

[0.0, 0.0]

In [95]:
sample_customers["preds"] = prediction.predictions

In [5]:
sample_customers

NameError: name 'sample_customers' is not defined

In [103]:
file_name = "predict_data/sample_customers.csv"
file_name.split(".")[0] + "_w_pred_" + file_name.split(".")[-1]

'predict_data/sample_customers_w_pred_csv'

In [ ]:
sample_customers.to_csv('predict_data/sample_customers_w_pred.csv', index=False)

# Getting endpoint

In [84]:
endpoint_id = "3280815153934761984"
endpoint_2 = Endpoint(endpoint_name = endpoint_id)

In [90]:
print(endpoint_2.resource_name)
print(endpoint_2.name)

projects/134756275535/locations/us-central1/endpoints/3280815153934761984
3280815153934761984


In [86]:
aiplatform.Endpoint(endpoint_name = endpoint_id, project = "")

resource name: projects/134756275535/locations/us-central1/endpoints/3280815153934761984

In [85]:
endpoint_2

resource name: projects/134756275535/locations/us-central1/endpoints/3280815153934761984

In [72]:
import requests

In [76]:
ENDPOINT_ID = "3280815153934761984"
PROJECT_ID = "united-impact-363519"
name = "Churn Model Endpoint"
endpoint_blah = requests.get(f"https://us-central1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-central1/endpoints/${ENDPOINT_ID}")

In [77]:
endpoint_blah

<Response [401]>

In [ ]:
curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-central1/endpoints/${ENDPOINT_ID}:predict \
-d "@${INPUT_DATA_FILE}"

# Debug

In [ ]:
def save_model(model, dest_bucket_uri, run_id):
        # Set up the GCS client and bucket
        client = storage.Client()

        bucket = client.get_bucket(dest_bucket_uri[5:])
        blob = bucket.blob(f'{run_id}/{model}')
        blob.upload_from_filename(f'{model}')

In [1]:
from google.cloud import storage

In [15]:
client = storage.Client()
dest_bucket_uri = "gs://churn_data_363519"
run_id = "churn-prediction-pipeline-20230406183239"
model = "sample_customers.csv"
model_folder = "lrc"
bucket = client.get_bucket(dest_bucket_uri[5:])
blob = bucket.blob(f'{run_id}/{model_folder}/{model}')
blob.upload_from_filename(f'{model}')

In [11]:
sample_customers

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
6182,34,3,25,6,2,2,2275.0,1468,807.0,0.757,4977,88,0.692,0.645
9746,49,1,36,2,3,2,19846.0,2162,17684.0,0.602,16908,122,0.694,0.109


In [14]:
import joblib

model = "lrc/sample_customers.joblib"

joblib.dump(sample_customers, model)

FileNotFoundError: [Errno 2] No such file or directory: 'lrc/sample_customers.joblib'